<a href="https://colab.research.google.com/github/walidMohmmed1999/Veritabani-Proje/blob/main/User_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2pMZTEk1QEdSzrIdRPPnEBkH62n_2LkmJ6emMAjy4yjdrb2KL")
public_url = ngrok.connect(5000).public_url

In [ ]:
print(public_url)

https://6c40-34-73-148-233.ngrok-free.app


In [ ]:
port_no = 5000

In [ ]:
!pip install pytz


In [ ]:
pip install openpyxl


In [ ]:
import torch
import cv2
from flask import Flask, request, jsonify, render_template, send_from_directory, Response, redirect, url_for, session
from pyngrok import ngrok
import threading
import os
import sqlite3
from datetime import datetime
import pytz
from werkzeug.security import generate_password_hash, check_password_hash
import csv


app = Flask(__name__)
app.secret_key = "gizli_anahtarınız"


ngrok.set_auth_token("2pMZTEk1QEdSzrIdRPPnEBkH62n_2LkmJ6emMAjy4yjdrb2KL")
port_no = 5000
public_url = ngrok.connect(port_no).public_url

UPLOAD_FOLDER = 'static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


DATABASE = "gecmis.db"

def init_db():
    """Veritabanını başlatın ve tabloları oluşturun (eğer yoksa)."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            label TEXT NOT NULL,
            date_uploaded TEXT NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

init_db()
print("Veritabanı başarıyla başlatıldı!")

model_path = "/content/drive/MyDrive/DentalRadiographsDataset/best_resnet18_full_model.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path, map_location=device)
model = model.to(device)
print("Model başarıyla yüklendi!")

TURKEY_TIMEZONE = pytz.timezone("Europe/Istanbul")

def classify_image(model, image_path):
    model.eval()


    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Görüntü bulunamadı: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)


    image = cv2.resize(image, (224, 224))

    image = (image / 127.5) - 1

    image_tensor = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float().to(device)

    with torch.no_grad():
        output = model(image_tensor)
        prediction = (output > 0.5).float().item()


    label = "Dikkat gerekiyor: Çürük tespit edildi." if prediction == 1 else "Harika haber! Bu dişler sağlıklı."
    return label

@app.route("/")
def home():
    if "user_id" in session:
        return render_template("index.html", username=session["username"])
    return redirect(url_for("login"))

@app.route("/signup", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form["username"]
        email = request.form["email"]
        password = request.form["password"]


        hashed_password = generate_password_hash(password)


        try:
            conn = sqlite3.connect(DATABASE)
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO users (username, email, password)
                VALUES (?, ?, ?)
            ''', (username, email, hashed_password))
            conn.commit()
            conn.close()
            return redirect(url_for("login"))
        except sqlite3.IntegrityError:
            return render_template("signup.html", error="Kullanıcı adı veya e-posta zaten mevcut.")
    return render_template("signup.html")

@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        email = request.form["email"]
        password = request.form["password"]

        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("SELECT id, username, password FROM users WHERE email = ?", (email,))
        user = cursor.fetchone()
        conn.close()

        if user and check_password_hash(user[2], password):
            session["user_id"] = user[0]
            session["username"] = user[1]
            return redirect(url_for("home"))
        return render_template("login.html", error="Geçersiz e-posta veya şifre.")
    return render_template("login.html")

@app.route("/logout")
def logout():
    session.clear()
    return redirect(url_for("login"))

@app.route("/upload", methods=["POST"])
def upload_file():
    if "user_id" not in session:
        return redirect(url_for("login"))

    if "file" not in request.files:
        return jsonify({"error": "Dosya sağlanmadı"}), 400
    file = request.files["file"]
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
    file.save(file_path)

    try:

        label = classify_image(model, file_path)


        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        current_time = datetime.now(TURKEY_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute('''
            INSERT INTO history (filename, label, date_uploaded)
            VALUES (?, ?, ?)
        ''', (file.filename, label, current_time))
        conn.commit()
        conn.close()


        image_url = f"{public_url}/static/uploads/{file.filename}"
        return jsonify({
            "message": f"Dosya {file.filename} başarıyla sınıflandırıldı!",
            "label": label,
            "image_url": image_url
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/dashboard")
def dashboard():
    if "user_id" not in session:
        return redirect(url_for("login"))

    """Kontrol panelini geçmişle görüntüleyin."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT id, filename, label, date_uploaded FROM history ORDER BY id DESC")
    rows = cursor.fetchall()
    conn.close()


    return render_template("dashboard.html", rows=rows, username=session["username"])

from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

@app.route("/download/excel")
def download_history_excel():
    """Generate and download the classification history as an Excel file."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT filename, label, date_uploaded FROM history ORDER BY id DESC")
    rows = cursor.fetchall()
    conn.close()

    wb = Workbook()
    ws = wb.active
    ws.title = "Classification History"


    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="007bff", end_color="007bff", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    border_style = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))


    headers = ["Dosya Adı", "Etiket", "Yükleme Tarihi"]
    for col_num, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col_num, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = border_style

    for row_num, row in enumerate(rows, 2):
        for col_num, cell_value in enumerate(row, 1):
            cell = ws.cell(row=row_num, column=col_num, value=cell_value)
            cell.border = border_style


            if row_num % 2 == 0:
                cell.fill = PatternFill(start_color="f4f4f9", end_color="f4f4f9", fill_type="solid")
            else:
                cell.fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")


    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[column_letter].width = max_length + 2


    excel_file_path = "classification_history.xlsx"
    wb.save(excel_file_path)


    return send_from_directory(directory=os.getcwd(), path=excel_file_path, as_attachment=True)



@app.route("/static/uploads/<filename>")
def serve_uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

def run_flask():
    print(f"Küresel bağlantıya erişmek için tıklayın: {public_url}")
    app.run(port=port_no, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()


Veritabanı başarıyla başlatıldı!


<ipython-input-9-dd9fa65dfeac>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Model başarıyla yüklendi!
Küresel bağlantıya erişmek için tıklayın: https://2214-34-73-148-233.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
def init_db():
    """Initialize the database and create tables if they don't exist."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()


    cursor.execute('''
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            label TEXT NOT NULL,
            date_uploaded TEXT NOT NULL
        )
    ''')


    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
    ''')

    conn.commit()
    conn.close()


init_db()
print("Database initialized with users table!")


Database initialized with users table!


In [ ]:
import sqlite3

conn = sqlite3.connect('history.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM history')
print(cursor.fetchall())
conn.close()


[(1, 'cate4_002.png', 'Dikkat gerekiyor: Çürük tespit edildi.', '2024-12-24 00:28:14'), (2, 'cate4_003.png', 'Dikkat gerekiyor: Çürük tespit edildi.', '2024-12-24 00:35:08')]


In [ ]:
import os
os.makedirs("templates", exist_ok=True)


In [ ]:
!ls


drive  gecmis.db  sample_data  static  templates


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Diş Radyografisi Sınıflandırıcısı</title>

    <link rel="stylesheet" href="/static/style.css">
</head>
<body>


    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard">Kontrol Paneli</a>
    </div>


    <div class="container">
        <h1>Çocuklar İçin Diş Radyografisi Sınıflandırıcısı</h1>
        <p>Yüklediğiniz diş radyografilerini analiz eder ve sonucu belirtir.</p>
        <form id="upload-form" method="post" enctype="multipart/form-data">
            <input type="file" name="file" id="file-input" required>
            <button type="submit">
                <i class="fa fa-cloud-upload"></i> Yükle
            </button>
        </form>
        <div id="result" class="result"></div>
        <div id="image-preview" class="image-preview"></div>
    </div>


    <div class="about-section">
        <h2>Hakkımızda</h2>
        <p>
            Hoş geldiniz! <strong>Çocuklar İçin Diş Radyografisi Sınıflandırıcısı</strong>, diş hekimliği profesyonellerine, radyografik görüntüleri analiz etmek ve çürük veya sağlıklı sınıflandırmalarını yapmak için tasarlanmış bir platformdur.
        </p>
        <p>
            Bu uygulama, derin öğrenme teknolojisini kullanarak teşhis doğruluğunu artırmayı ve hasta bakımını iyileştirmeyi amaçlar. Amacımız, kullanıcı dostu bir arayüz ile verimli bir çözüm sunmaktır.
        </p>
        <p>
            <strong>Misyonumuz:</strong> Diş hekimlerini AI destekli araçlarla güçlendirerek daha doğru teşhisler yapmalarını sağlamak ve mükemmel hasta bakımı sunmalarına destek olmaktır.
        </p>
    </div>


    <footer>
        <p>&copy; 2024 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı | Developed by [Walid Mohmmed]</p>
    </footer>


    <script>
        const form = document.getElementById('upload-form');
        const resultDiv = document.getElementById('result');
        const imagePreview = document.getElementById('image-preview');

        form.onsubmit = async (e) => {
            e.preventDefault();
            const formData = new FormData(form);
            resultDiv.textContent = "İşleniyor...";
            imagePreview.innerHTML = "";

            try {
                const response = await fetch('/upload', {
                    method: 'POST',
                    body: formData
                });
                const data = await response.json();
                resultDiv.textContent = `Sonuç: ${data.label}`;


                if (data.image_url) {
                    const img = document.createElement('img');
                    img.src = data.image_url;
                    img.alt = "Uploaded Image";
                    img.style.maxWidth = "100%";
                    imagePreview.appendChild(img);
                }
            } catch (error) {
                resultDiv.textContent = "Bir hata oluştu!";
            }
        };
    </script>
</body>
</html>


Overwriting templates/index.html


In [ ]:
%%writefile templates/dashboard.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Kontrol Paneli - Sınıflandırma Geçmişi</title>
    <link rel="stylesheet" href="/static/style.css">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>

    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard">Kontrol Paneli</a>
    </div>

    <div class="container">
        <h2>Yüklenen Görüntülerin Geçmişi</h2>
        <table>
            <thead>
                <tr>
                    <th>Dosya Adı</th>
                    <th>Etiket</th>
                    <th>Yükleme Tarihi</th>
                </tr>
            </thead>
            <tbody>
                {% for row in rows %}
                <tr>
                    <td>{{ row[1] }}</td>
                    <td>{{ row[2] }}</td>
                    <td>{{ row[3] }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>



    <div class="container">
    <h2>Geçmişi İndir</h2>
    <a href="/download/excel" class="download-button excel-download">Excel Formatında İndir</a>
</div>



    <footer>
        <p>&copy; 2024 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı</p>
    </footer>


    <script>

        fetch('/stats')
            .then(response => response.json())
            .then(data => {
                document.getElementById('chart-spinner').style.display = 'none';
                document.getElementById('classificationChart').style.display = 'block';
                const ctx = document.getElementById('classificationChart').getContext('2d');
                new Chart(ctx, {
                    type: 'doughnut',
                    data: {
                        labels: ['Sağlıklı', 'Çürük'],
                        datasets: [{
                            label: 'Sınıflandırma İstatistikleri',
                            data: [data.healthy, data.caries],
                            backgroundColor: [
                                'rgba(75, 192, 192, 0.6)',
                                'rgba(255, 99, 132, 0.6)'
                            ],
                            borderColor: [
                                'rgba(75, 192, 192, 1)',
                                'rgba(255, 99, 132, 1)'
                            ],
                            borderWidth: 1
                        }]
                    },
                    options: {
                        responsive: true,
                        plugins: {
                            legend: {
                                position: 'top',
                            },
                            title: {
                                display: true,
                                text: 'Sınıflandırma Özeti'
                            },
                            tooltip: {
                                callbacks: {
                                    label: function(tooltipItem) {
                                        const dataset = tooltipItem.dataset;
                                        const total = dataset.data.reduce((prev, curr) => prev + curr, 0);
                                        const value = dataset.data[tooltipItem.dataIndex];
                                        const percentage = ((value / total) * 100).toFixed(2);
                                        return `${dataset.labels[tooltipItem.dataIndex]}: ${value} (${percentage}%)`;
                                    }
                                }
                            }
                        }
                    }
                });
            })
            .catch(error => {
                document.getElementById('chart-spinner').style.display = 'none';
                console.error('Grafik verileri alınırken hata oluştu:', error);
            });
    </script>
</body>
</html>


Writing templates/dashboard.html


In [ ]:
%%writefile templates/signup.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Kayıt Ol</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>

    <!-- Form Konteyneri -->
    <div class="form-container">
        <h2>Hesap Oluştur</h2>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="POST" action="/signup">
            <input type="text" name="username" placeholder="Kullanıcı Adı" required>
            <input type="email" name="email" placeholder="E-posta" required>
            <input type="password" name="password" placeholder="Şifre" required>
            <button type="submit">Kayıt Ol</button>
        </form>
        <p class="form-footer">Zaten bir hesabınız var mı? <a href="/login">Giriş Yapın</a>.</p>
    </div>
</body>
</html>


Overwriting templates/signup.html


In [ ]:
%%writefile templates/login.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Giriş Yap</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>


    <div class="form-container">
        <h2>Tekrar Hoş Geldiniz</h2>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="POST" action="/login">
            <input type="email" name="email" placeholder="E-posta" required>
            <input type="password" name="password" placeholder="Şifre" required>
            <button type="submit">Giriş Yap</button>
        </form>
        <p class="form-footer">Hesabınız yok mu? <a href="/signup">Buradan kayıt olun</a>.</p>
    </div>
</body>
</html>


Overwriting templates/login.html


In [ ]:
mkdir static


mkdir: cannot create directory ‘static’: File exists


In [ ]:
%%writefile static/style.css
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f9;
    margin: 0;
    padding: 0;
}

/* Navbar styling */
.navbar {
    background-color: #007bff;
    color: #fff;
    padding: 15px;
    text-align: center;
    font-size: 18px;
    font-weight: bold;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    position: fixed;
    top: 0;
    width: 100%;
    z-index: 1000;
    display: flex;
    justify-content: center;
    gap: 15px;
}

.navbar a {
    color: #fff;
    text-decoration: none;
    padding: 10px 15px;
    border-radius: 5px;
    transition: all 0.3s ease;
    font-size: 16px;
}

.navbar a:hover {
    background-color: #0056b3;
    color: #f4f4f9;
    transform: scale(1.05);
}

/* Dashboard table styling */
table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
}

table th, table td {
    border: 1px solid #ddd;
    padding: 10px;
    text-align: center;
    font-size: 14px;
}

table th {
    background-color: #007bff;
    color: #fff;
    font-weight: bold;
    font-size: 16px;
}

table tr:nth-child(even) {
    background-color: #f9f9f9;
}

table tr:hover {
    background-color: #f1f1f1;
}


#classificationChart {
    margin: 20px auto;
    max-width: 80%;
    padding: 20px;
    background: #fff;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}


.download-button {
    display: inline-block;
    background-color: #007bff;
    color: #fff;
    padding: 10px 20px;
    margin-top: 20px;
    border-radius: 5px;
    text-decoration: none;
    font-size: 16px;
    font-weight: bold;
    transition: all 0.3s ease;
}

.download-button:hover {
    background-color: #0056b3;
    transform: scale(1.05);
}


.form-container {
    max-width: 400px;
    margin: 120px auto;
    background: #fff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.form-container h2 {
    color: #333;
    margin-bottom: 20px;
    font-size: 24px;
}

.form-container .error {
    color: red;
    font-size: 14px;
    margin-bottom: 15px;
    text-align: center;
}


.form-container input[type="text"],
.form-container input[type="email"],
.form-container input[type="password"] {
    width: 90%;
    padding: 10px;
    margin: 10px 0;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-size: 16px;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
    transition: all 0.2s ease-in-out;
}

.form-container input[type="text"]:focus,
.form-container input[type="email"]:focus,
.form-container input[type="password"]:focus {
    outline: none;
    border-color: #007bff;
    box-shadow: 0 0 8px rgba(0, 123, 255, 0.5);
}


.form-container button {
    background-color: #007bff;
    color: #fff;
    border: none;
    padding: 10px 20px;
    border-radius: 5px;
    font-size: 16px;
    cursor: pointer;
    transition: all 0.3s;
}

.form-container button:hover {
    background-color: #0056b3;
    transform: scale(1.05);
}


.form-container .form-footer {
    margin-top: 15px;
    font-size: 14px;
}

.form-container .form-footer a {
    color: #007bff;
    text-decoration: none;
    font-weight: bold;
}

.form-container .form-footer a:hover {
    text-decoration: underline;
}


.error {
    color: red;
    font-size: 14px;
    margin-bottom: 15px;
    text-align: center;
}


.container {
    max-width: 800px;
    margin: 100px auto 50px auto;
    background: #fff;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    text-align: center;
}

/* Hero Section */
.container h1 {
    color: #333;
    font-size: 28px;
    margin-bottom: 10px;
}

.container p {
    color: #666;
    font-size: 16px;
    margin-bottom: 20px;
}

/* Input styling */
input[type='file'] {
    display: block;
    margin: 20px auto;
    padding: 5px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-size: 16px;
    cursor: pointer;
    max-width: 90%;
}


.result {
    margin-top: 20px;
    font-size: 20px;
    color: #007bff;
    font-weight: bold;
}


.image-preview img {
    max-width: 100%;
    margin-top: 20px;
    border: 1px solid #ccc;
    border-radius: 5px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}


.about-section {
    background-color: #fff;
    padding: 20px;
    margin: 30px auto;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.about-section h2 {
    color: #333;
    margin-bottom: 15px;
}

.about-section p {
    color: #555;
    font-size: 14px;
    line-height: 1.6;
}


footer {
    background-color: #f4f4f9;
    color: #666;
    text-align: center;
    padding: 10px 0;
    margin-top: 20px;
    font-size: 14px;
    border-top: 1px solid #ddd;
}




.download-button {
    display: inline-block;
    margin: 10px auto;
    padding: 12px 20px;
    background-color: #007bff; -
    color: #fff; -
    font-size: 16px;
    font-weight: bold;
    text-align: center;
    text-decoration: none;
    border-radius: 8px;
    transition: all 0.3s ease;
    border: 2px solid #0056b3;
    cursor: pointer;
}

.download-button:hover {
    background-color: #0056b3;
    border-color: #004080;
    transform: scale(1.05);
}


.excel-download {
    background-color: #28a745;
    border-color: #1c7430;
}

.excel-download:hover {
    background-color: #1c7430;
    border-color: #155724;
    transform: scale(1.05);
}


button {
    background: linear-gradient(45deg, #007bff, #0056b3);
    color: #fff;
    padding: 12px 25px;
    border: none;
    border-radius: 25px;
    font-size: 16px;
    font-weight: bold;
    cursor: pointer;
    display: inline-flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
    transition: all 0.3s ease-in-out;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

button:hover {
    background: linear-gradient(45deg, #0056b3, #007bff);
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.2);
    transform: scale(1.05);
}

button:active {
    transform: scale(0.95);
    box-shadow: 0 3px 6px rgba(0, 0, 0, 0.2);
}

button i {
    font-size: 18px;
}



Overwriting static/style.css
